In [26]:
import pandas as pd
import defusedxml.ElementTree as ET
import re

## Školská zařízení (MŠ, ZŠ, střední aj.) - scrap kapacit

In [58]:
tree = ET.parse(r'data\open_data_MSMT\vrejcelk.xml')
root = tree.getroot()

In [59]:
skoly_data = []
zarizeni_info_data = []
zarizeni_misto_data = []
skola_obor_data = []

for ps in root.findall('.//PravniSubjekt'):
    # TAB "skoly" a "zarizeni_info"
    ico = ps.find('ICO').text
    nazev = ps.find('Reditelstvi').find('RedPlnyNazev').text
    for sz in ps.find('SkolyZarizeni').findall('SkolaZarizeni'):
        izo = sz.find('IZO').text
        typZarizeni = sz.find('SkolaPlnyNazev').text
        zarizeniDruhTyp = sz.find('SkolaDruhTyp').text
        zarizeniKapacita = sz.find('SkolaKapacita').text
        kapacitaJednotka = sz.find('SkolaKapacitaJednotka').text
        jazyk = sz.find('SkolaJazyk').text

        raw_skoly = {'IZO' : izo,  
                    'ZarizeniTyp': typZarizeni, 
                    'Kapacita': zarizeniKapacita,
                    'KapacitaJednotka': kapacitaJednotka,           
                    }
        raw_zarizeni_info = {
                    'ICO': ico, 
                    'ReditelstviNazev' : nazev, 
                    'ZarizeniDruhTyp' : zarizeniDruhTyp, 
                    'jazyk' : jazyk, 
                    'IZO' : izo,
                    }
        skoly_data.append(raw_skoly)
        zarizeni_info_data.append(raw_zarizeni_info)

        # TAB "zarizeni_misto"
        for smvc in sz.find('SkolaMistaVykonuCinnosti').findall('SkolaMistoVykonuCinnosti'):
            druh = smvc.find('MistoDruhTyp').text # u středních škol např. rozlišení gymnázií, u MŠ specializace

            # extrahování PSČ
            def extract_psc(adresa_text):
                if adresa_text:
                    PSC_match = re.search(r'\d{3} ?\d{2}', adresa_text)
                    return PSC_match.group() if PSC_match else adresa_text
                return None
            PSC = extract_psc(smvc.find('MistoAdresa2').text)        
            PSC2 = extract_psc(smvc.find('MistoAdresa3').text)

                        
            raw_zarizeni_misto = {
                    'druh' : druh,
                    'adresa2': PSC,
                    'adresa3': PSC2,
                    'IZO' : izo,
                   }
            zarizeni_misto_data.append(raw_zarizeni_misto)
            
        # TAB "skola_obor"
        if sz.find('SkolaOboryVzdelani') is not None:
            for sov in sz.find('SkolaOboryVzdelani').findall('SkolaOborVzdelani'):
                oborNazev = sov.find('OborNazev').text
                delkaVzdelavani = sov.find('DelkaVzdelavani').text
                oborKapacita = sov.find('OborKapacita').text
                oborDobihajici = sov.find('OborDobihajici').text
                # print(oborNazev)
    
                raw_skola_obor = {
                    'OborNazev' : oborNazev,
                    'DelkaVzdelavani': delkaVzdelavani,
                    'OborKapacita' : oborKapacita,
                    'OborDobihajici' : oborDobihajici,
                    'IZO' : izo,
                   }
                skola_obor_data.append(raw_skola_obor)

### Školy (kapacita) data -> dataframe dle ERD

Převedení raw dat do dataframů, následné vytvoření sloupců 'PSC' a 'obec' a jejich vyplnění dle nadefinovaných funkcí.\
**makePsc2** -> pokud se ve sloupcích adresa2 a adresa3 nachází PSC, vyextrahuje se k pozdějšímu napárování krajů (přes okresy)\
**makeObec** -> jestliže jsme získali PSC ze sloupce adresa3, pak do obce se nám zapíše adresa2+PSC jako unikátní kombinace identifikátoru (adresa3 mohla být none i jiná str informace), pokud adresa3 nebo PSC nebylo vyplněno, do obce se nám zapíše adresa2, \
Chybějící hodnoty v PSC nahradíme nulou a převedeme hodnoty na int


In [60]:
skoly = pd.DataFrame(skoly_data)

zarizeni_info = pd.DataFrame(zarizeni_info_data)
zarizeni_info.insert(loc=0, column='ID', value=range(1, len(zarizeni_info_data) + 1))
    
zarizeni_misto = pd.DataFrame(zarizeni_misto_data)
zarizeni_misto.insert(loc=0, column='ID', value=range(1, len(zarizeni_misto) + 1))
    
skola_obor = pd.DataFrame(skola_obor_data)
skola_obor.insert(loc=0, column='ID', value=range(1, len(skola_obor_data) + 1))
  
def makePsc(row):
    adresa2 = row['adresa2']
    adresa3 = row['adresa3']
    if adresa3:
        psc_match = re.search(r'\d{3} ?\d{2}', adresa3)
        if psc_match:
            return psc_match.group().replace(' ', '')
    elif adresa2:
        psc_match = re.search(r'\d{3} ?\d{2}', adresa2)
        if psc_match:
            return psc_match.group().replace(' ', '')
            
def makeObec(row):
    PSC_obec = row['PSC']
    adresa2 = row['adresa2']
    adresa3 = row['adresa3']
    if PSC_obec:
        if adresa3:
            psc_match = re.search(r'\d{3} ?\d{2}', adresa3)
            if psc_match.group().replace(' ', '') == PSC_obec:
                return adresa2 + ',' + PSC_obec
        return adresa2
    else:
        return adresa2
        

zarizeni_misto['PSC'] = zarizeni_misto.apply(
    makePsc, axis=1) 

zarizeni_misto['obec'] = zarizeni_misto.apply(   
    makeObec, axis=1)

# null hodnoty v 'PSC'nahrazeny číslem 0, následně celý sloupe převeden na int
zarizeni_misto['PSC'] = zarizeni_misto['PSC'].fillna(0)
zarizeni_misto = zarizeni_misto.astype({'PSC':'int'})
# len(zarizeni_misto[zarizeni_misto['PSC'] == 0]) # kde chybí psč máme alespoň identifikátor obec
zarizeni_misto['IZO'].value_counts()

IZO
181064791    108
110029534     56
108012263     46
161100473     41
108027384     35
            ... 
181144841      1
181144913      1
181144921      1
181145057      1
181145227      1
Name: count, Length: 27188, dtype: int64

Předpříprava převodníku (z GitHubu) PSČ -> kraj

In [38]:
psc_okres = pd.read_csv(r'data\adresa\psc2okres.csv') # obsahuje duplicity, a celkem uniq 77 okresů
okres_kraj = pd.read_csv(r'data\adresa\okres2kraj.csv') # pouze unikátní hodnoty, míň okresu, než má psc okresů, 82 okresů
psc_okres = psc_okres.drop_duplicates() # dropnutí duplikátu z cca 17tis na cca 3tis záznamů

Spojení tabulek k převodu - PSČ-Kraj, spojení přes okres.\
Vyhledání problematických PSC, které figutují ve vícero krajích (může zapříčinit zkreslení výsledků)\
**make_kraj** -> do vytvořeného sloupce 'Kraj' napárujeme dle PSC kraj pouze tehdy, pokud nám PSC nefiguruje ve vícero krajích 


In [39]:
# vytvoreni nové tabulky -> dle PSČ zjistime pravděpodobně kraj
psc_kraj = okres_kraj.merge(psc_okres, how='right', on='Okres')

#kde není vyplněn kraj, přehodíme tam okres - jedná se např. o Hl. město Prahu, aj.
psc_kraj['Kraj'] = psc_kraj['Kraj'].fillna(psc_kraj['Okres'])

# len(psc_kraj['PSC'].unique()) #3007, 3165 je celkový počet, tzn. 158 PSČ celkem figuruje ve vícero krajích

# psc figurující ve vícero krajích 
psc_duplicates = psc_kraj.groupby('PSC')['Kraj'].nunique()

shared_psc = psc_duplicates[psc_duplicates > 1].index
shared_kraje = psc_kraj[psc_kraj['PSC'].isin(shared_psc)]
shared_kraje_set = set(shared_kraje['PSC'])
shared_kraje_set_df = pd.DataFrame(shared_kraje_set)
shared_kraje_set_df = shared_kraje_set_df.rename(columns={0: 'PSC'})

def makeKraj(row):
    zarizeni_psc = row['PSC']
    
    if zarizeni_psc not in shared_kraje_set_df['PSC'].values:
        matching_row = psc_kraj[psc_kraj['PSC'] == zarizeni_psc]

        if not matching_row.empty:
            return matching_row['Kraj'].values[0]       
    else:
        return None
        
zarizeni_misto['Kraj'] = zarizeni_misto.apply(
    makeKraj, axis=1) 

Filtrování zařízení bez přiřazených krajů, následná extrakce, prohnání výsledků AI, náhodná ruční kontrola výsledků, nahrání a následné přiřazení výsledků k zarizeni_misto.\
Jedná se o zařízení, které buďto vůbec nemají PSČ (ale mají jiné str informace) nebo PSČ mají, ale to je přiřazeno ke dvěma krajům.

In [40]:
zarizeni_misto_filtered = zarizeni_misto[zarizeni_misto['Kraj'].isnull()]
zarizeni_misto_filtered

uniq_obec = zarizeni_misto_filtered['obec'].unique() # snízení záznamů na 1/7
uniq_obec = pd.DataFrame(uniq_obec).rename(columns={0: 'Kraj'})
# uniq_obec.to_excel(r'output_files\zarizeni_misto_filtered1.xlsx') #následně hodnoty zkopírovány a vloženy do Copilotu (Gemini ani ChatGPT nefungovali)

# zarizeni_misto['Kraj'].value_counts()
# uniq_obec['Kraj'].value_counts()
# zarizeni_misto.info()

Import dohledaných a hlavně problematických dat pomocí MS Copilot, převedení na df, následné napárovnání na zarizeni_misto

In [41]:
import data.adresa.kraj_copilot as krajc
location = krajc.locations
location_df = pd.DataFrame(list(location.items()), columns=['Location', 'Kraj'])
location_df

def make_missingKraj(row):
    zarizeni_obec = row['obec']
    zarizeni_kraj = row['Kraj']
    
    if pd.isnull(zarizeni_kraj):
        match = location_df[location_df['Location'] == zarizeni_obec]
        if not match.empty:
            return match['Kraj'].values[0]
    return zarizeni_kraj
   
zarizeni_misto['Kraj'] = zarizeni_misto.apply(
    make_missingKraj, axis=1) 

Závěrečná kontrola - zarizeni_misto nemá v kraji nulové hodnoty + ve výčtu uniq hodnot jsou pouze kraje. 

In [42]:
zarizeni_misto['Kraj'] = zarizeni_misto['Kraj'].replace(['Frýdek Místek', 'Třinec'], 'Moravskoslezský kraj')
zarizeni_misto[zarizeni_misto['Kraj'].isnull()] # finální ověření, že žádný kraj nechybí
zarizeni_misto['Kraj'].value_counts()
# zarizeni_misto.info()

Kraj
Středočeský kraj        5263
Jihomoravský kraj       4653
Moravskoslezský kraj    4140
Hlavní město Praha      3335
Olomoucký kraj          2797
Ústecký kraj            2645
Jihočeský kraj          2333
Královéhradecký kraj    2293
Zlínský kraj            2270
Kraj Vysočina           2173
Pardubický kraj         2027
Plzeňský kraj           1920
Liberecký kraj          1610
Karlovarský kraj        1074
Name: count, dtype: int64

## DS, MŠ + PSZŠSpeciální

### MŠ + PSZŠSpeciální (kapacita) TAB

In [43]:
materske_skoly_zarizeni = ["Mateřská škola", "Mateřská škola (lesní mateřská škola)", "Mateřská škola - Przedszkole", "Mateřská škola s p.j.v.", "Přípravný stupeň základní školy speciální"]
materske_skoly = skoly[skoly['ZarizeniTyp'].isin(materske_skoly_zarizeni)]
materske_skoly = materske_skoly.astype({'Kapacita':'int'})
# materske_skoly.info()
# materske_skoly['Kapacita'].sum()
materske_skoly

,IZO,ZarizeniTyp,Kapacita,KapacitaJednotka
0,049625918,Mateřská škola,54,dítě
2,107500884,Mateřská škola,70,dítě
4,110034384,Mateřská škola,6,dítě
6,060437171,Mateřská škola,130,dítě
8,110035585,Mateřská škola,18,dítě
...,...,...,...,...
27171,181145014,Mateřská škola (lesní mateřská škola),15,dítě
27177,181145499,Mateřská škola (lesní mateřská škola),15,dítě
27181,181145871,Mateřská škola,44,dítě
27187,181146878,Mateřská škola (lesní mateřská škola),16,dítě


**Je potřeba provést kontrolu, zda jedno MŠ(školské) zařízení nefiguruje ve vícero krajích. Což se bohužel potvrdilo.** Přesné kapacity každého zařízení se nedají dohledat na internetu.\
Nejčastěji se však jedná o rozmělnění v rámci Hlavní město Praha a Středočeský Kraj. Do počtu zařízení pak vede Praha, proto výsledky v tomto konktrétním případě vždy přiřadíme právě Praze.\
V případě jiné kombinaci sdílení krajů pro IZO zařízení vždy přiřadíme kapacity jinému kraji, než je Hlavní město Praha a Středočeský Kraj. Případně přiřadíme první vyhledaný kraj. Jedná se o desítky až nižší stovky míst zkreslení dat v rámci regionů se . \
Prozatím se neobjevil případ, kdy by pro jedno IZO zařízení bylo přiřazování více krajů než dva. **Manuální kontrola (zakomentováno níže).**

In [44]:
ms_duplicates = zarizeni_misto[zarizeni_misto['IZO'].isin(materske_skoly['IZO'])].groupby('IZO')['Kraj'].nunique()
shared_ms_kraje = ms_duplicates[ms_duplicates > 1].index
# shared_ms_kraje = ms_duplicates[ms_duplicates > 2].index # pro kontrolu, zda zařízení nefigurující ve vícero, než 2 krajích

# shared_ms_kraje
zarizeni_misto[zarizeni_misto['IZO'].isin(shared_ms_kraje)] # dataframe, který zobrazí zařízení sdílená s vícero kraji 
# materske_skoly[materske_skoly['IZO'].isin(shared_ms_kraje)] # dataframe, který nám zobrazí kapacity zařízení, která náleží vícero krajům
# materske_skoly[materske_skoly['IZO'].isin(shared_ms_kraje)]['Kapacita'].sum() # přesný výpočet kapacity celkem, která se dělí mezi vícero krajů (nejvíc jí patří do Prahy)
materske_skoly[materske_skoly['IZO'] == '181142180'] # jediné zařízení v rámci mš, které vůbec nemá vyplněné místo

,IZO,ZarizeniTyp,Kapacita,KapacitaJednotka
27088,181142180,Mateřská škola,16,dítě


spojit tabulky, opodmínkovat to, že ty, keré mají izo in shared_ms_kraje

In [45]:
def assignKraj(row):
    MS_IZO = row['IZO']
    
    if MS_IZO not in shared_ms_kraje:
        matching_row = zarizeni_misto[zarizeni_misto['IZO'] == MS_IZO]
        if not matching_row.empty:
            return matching_row['Kraj'].values[0] 
    elif MS_IZO in shared_ms_kraje:
        matching_row = zarizeni_misto[zarizeni_misto['IZO'] == MS_IZO]
        kraje = matching_row['Kraj'].unique()

        # Podmínka pro 'Hlavní město Praha' a 'Středočeský kraj'
        if 'Hlavní město Praha' in kraje and 'Středočeský Kraj' in kraje:
            return 'Hlavní město Praha'

        # Podmínka pro 'Hlavní město Praha' a jakýkoliv jiný kraj
        elif 'Hlavní město Praha' in kraje and len(kraje) > 1:
            other_kraje = [k for k in kraje if k != 'Hlavní město Praha']
            return other_kraje[0]  # Vrací první jiný kraj než 'Hlavní město Praha'

        # Podmínka pro 'Středočeský kraj' a jakýkoliv jiný kraj
        elif 'Středočeský kraj' in kraje and len(kraje) > 1:
            other_kraje = [k for k in kraje if k != 'Středočeský kraj']
            return other_kraje[0]  # Vrací první jiný kraj než 'Středočeský kraj'
                    
        else:
            return matching_row['Kraj'].values[0] # Vrať první kraj, pokud podmínky neplatí.
            
    else:
        return None
        
materske_skoly['Kraj'] = materske_skoly.apply(
    assignKraj, axis=1) 

In [46]:
# materske_skoly[materske_skoly['Kraj'].isnull()] #IZO 181142180 -> mš vůbec nemá zarizeni_misto

In [69]:
materske_skoly.loc[materske_skoly['IZO'] == '181142180', 'Kraj'] = 'Jihočeský kraj'
materske_skoly.info()
materske_skoly['Kraj'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Index: 5487 entries, 0 to 27188
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   IZO               5487 non-null   object
 1   ZarizeniTyp       5487 non-null   object
 2   Kapacita          5487 non-null   int64 
 3   KapacitaJednotka  5482 non-null   object
 4   Kraj              5487 non-null   object
dtypes: int64(1), object(4)
memory usage: 257.2+ KB


Kraj
Středočeský kraj        840
Jihomoravský kraj       694
Moravskoslezský kraj    480
Hlavní město Praha      448
Olomoucký kraj          391
Ústecký kraj            368
Jihočeský kraj          342
Zlínský kraj            333
Pardubický kraj         328
Královéhradecký kraj    323
Kraj Vysočina           292
Plzeňský kraj           281
Liberecký kraj          238
Karlovarský kraj        129
Name: count, dtype: int64

### DětskéSkupinky (realtime kapacita) TAB

In [2]:
import data.web_scrap.web_scrap_DS as DS_scrap
DS_tabulka = DS_scrap.DS_tab

In [3]:
DS_tabulka = DS_tabulka.astype({'Kapacita':'int'})
# DS_tabulka.info()
DS_tabulka

,ZarizeniTyp,Kapacita,Kraj
0,Dětská skupinka,4633,Hlavní město Praha
1,Dětská skupinka,1311,Jihočeský kraj
2,Dětská skupinka,4075,Jihomoravský kraj
3,Dětská skupinka,262,Karlovarský kraj
4,Dětská skupinka,1147,Královéhradecký kraj
5,Dětská skupinka,782,Liberecký kraj
6,Dětská skupinka,2200,Moravskoslezský kraj
7,Dětská skupinka,1386,Olomoucký kraj
8,Dětská skupinka,1092,Pardubický kraj
9,Dětská skupinka,1077,Plzeňský kraj


### **Předškolní vzdělávání a péče TAB (kapacita a realtime kapacita)**

In [50]:
predskolniVP = pd.concat([materske_skoly, DS_tabulka], ignore_index=True, sort=False)
predskolniVP.info()
predskolniVP

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5501 entries, 0 to 5500
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   IZO               5487 non-null   object
 1   ZarizeniTyp       5501 non-null   object
 2   Kapacita          5501 non-null   int64 
 3   KapacitaJednotka  5482 non-null   object
 4   Kraj              5501 non-null   object
dtypes: int64(1), object(4)
memory usage: 215.0+ KB


,IZO,ZarizeniTyp,Kapacita,KapacitaJednotka,Kraj
0,049625918,Mateřská škola,54,dítě,Hlavní město Praha
1,107500884,Mateřská škola,70,dítě,Hlavní město Praha
2,110034384,Mateřská škola,6,dítě,Hlavní město Praha
3,060437171,Mateřská škola,130,dítě,Hlavní město Praha
4,110035585,Mateřská škola,18,dítě,Hlavní město Praha
...,...,...,...,...,...
5496,NaN,Dětská skupinka,1077,NaN,Plzeňský kraj
5497,NaN,Dětská skupinka,4106,NaN,Středočeský kraj
5498,NaN,Dětská skupinka,1595,NaN,Ústecký kraj
5499,NaN,Dětská skupinka,579,NaN,Kraj Vysočina


In [51]:
predskolniVP.to_csv(r'output_files\predskolniVP.csv', mode='w')

## Porodnost 2018-2023 TAB (neměnná data z webu ČSÚ)

Pro přesnost bychom od porodnosti měli správně odečítat počet zemřelých. Vzhledem k nedostatečně podrobným datům s tímto údajem pracovat nebudeme, jedná se ale o nižší stovky ve věku 0 - 14 let., podrobnější členění lze nalézst do 1roku, případně od 0-5let, což je pro nás stále nedostatečně podrobné.

Vzhledem k málo podrobným datům i v rámci porodnosti (roční souhr za každý kraj nám výsledek dostupnosti výchovných a školských zařízení zkreslí 8měsíců). Je nutno si tedy manuálně vytáhnout tabulku porodnosti z VD ČSU a následně si data, co nejpřesněji dopočítat. Prakticky se nejpodrobnější data dají získat a dopočítat za jednotlivá čtvrtletí, pro naše účely proto:

**(leden až září (n) - 1.pololetí (n)) / 3 = 1/3 3.čtvrtletí (n)**

**porodnost do začátku školního roku = 1.pololetí(n) + 2/3 3.čtvrtletí (n)**

**porodnost od začátku školního roku, která se přičte k 8.měsícům porodnosti roku n+1 -> (1/3 3.čtvrtletí + rok (n) - leden až září (n)) + 1.pololetí(n+1) + 2/3 3.čtvrtletí (n+1)** -> od dovršení 3 let do 31.8.x (respektive 31.8.(n+3) vzniká nárok nastoupit do školky, proto by tomu měly odpovídat regionální kapacity MŠ

V počtech dětí při dělení zaokrouhluejme na celá čísla nahoru.\
\
Porodnost (živě narozených) zahrnuje i narození dětí s jiným občanstvím. Prozatím se nepodařilo najít dostatečně podrobná data(porovnání krajů) na to, tento údaj zahrnout do dat. Lze dohledat počet "Živě narozených cizinců", ovšem jen k celé ČR. Od roku 1995 se podíl cizinců na živě narozených v ČR (v %) postupně zvyšuje z 0,7 na 6,4 (2022), na 8,3 (2023).
více infa: https://csu.gov.cz/pocet-cizincu-demograficke-udalosti?pocet=10&start=0&podskupiny=291&razeni=-datumVydani


### **Porodnost podrobná pro školní roky 2019/2020-2023/2024 dle regionů**

V excelu dopočítané hodnoty, význam sloupců následující: \
porodnost do 2019/2020 -> děti, které se narodily (od září do prosince r. 2019) + (od ledna do srpna r.2020), tzn. jedná se o děti, které o 3 roky později mají již nárok na předškolní vzdělávání, tj ve školním roce 2022/2023 by měli mít možnost nastoupit do předškolního vzdělávání.
porodnost do 2020/2021 -> ve školním roce 2023/2024 by měli mít možnost nastoupit do předškolního vzdělávání.

In [52]:
porodnost_podr = (pd.read_excel(r'data\porodnost\porodnost_podr_uprav.xlsx', sheet_name=0, header=6, index_col=0, usecols='B, AO:AS', nrows=14))
porodnost_podr

,porodnost do 2023/2024,porodnost do 2022/2023,porodnost do 2021/2022,porodnost do 2020/2021,porodnost do 2019/2020
Hlavní město Praha,12869,14178,14967,14819,15147
Jihočeský kraj,5559,6335,6709,6479,6752
Jihomoravský kraj,11133,12519,12990,12935,13489
Karlovarský kraj,2216,2436,2678,2742,2775
Kraj Vysočina,4615,5192,5478,5416,5462
Královéhradecký kraj,4770,5175,5499,5567,5586
Liberecký kraj,3703,4103,4442,4557,4695
Moravskoslezský kraj,9975,11243,11910,11871,12103
Olomoucký kraj,5523,6165,6456,6538,6481
Pardubický kraj,4671,5180,5399,5543,5609


In [53]:
porodnost_podr.to_csv(r'output_files\porodnost_podr.csv', mode='w')

### Porodnost tab, málo podrobná, prozatím odložená

In [42]:
# from web_scrap_porodnost import porodnost_tabulka

In [43]:
# porodnost_tab = porodnost_tabulka.astype({"PocetNarozenychDeti":'int'}) 
# porodnost_tabCR = porodnost_tab[porodnost_tab['Kraj'] == 'Česká republika']
# porodnost_tab_kraje = porodnost_tab[porodnost_tab['Kraj'] != 'Česká republika']
# # porodnost_tab_kraje
# # porodnost_tabCR[porodnost_tabCR['Rok'].isin([2021, 2020, 2019, 2018])]['PocetNarozenychDeti'].sum()
# # porodnost_tab_kraje.info()
# porodnost_tab_kraje

In [44]:
# porodnost_tab_kraje.to_csv('porodnost.csv', mode='w')

## ČSÚ: Školy a školská zařízení - reálný počet žáků

### Pro rozbor ZŠ, Do 1třídy nastupuje více dětí, než kolik mělo odklad. Vzhledem k povinnému poslednímu ročníku v MŠ se pravděpodobně jedná o imigraci?

In [45]:
# csu = pd.read_excel(r'D:\git\skolstvi_projekt\CSU\230042230202.xlsx', sheet_name=['2.2.17', '2.2.18'], header = 3, index_col=0, usecols='A, H:L')
# # 2.2.17 -> počet žáků nově přijatých do 1. ročníku celkem
# # 2.2.18 -> počet žáků 7letých a starších nově přijatých do 1. ročníku, podmnožina 2.2.17
# # pro pozdější užití, na kartě 2.2.20 -> žáci opakující ročník, údaj "z toho v 1.ročníku" nutno PŘIČÍST k údaji z 2.2.17, 
# #žel tento údaj nění formátovaný v časocé řadě, nutno pro každý ročník explicitně stáhnout samostatný xlsx soubor

# nPrijatiZS = pd.DataFrame(csu['2.2.17'])
# nPrijatiZS_7plus = pd.DataFrame(csu['2.2.18'])
# # nPrijatiZS
# # nPrijatiZS_7plus

### Pro rozbor MŠ

Jedná se o oficiální data, kolik dětí docházelo do zařízení. 

In [54]:
def get_MS_dochazka(path, sheet='1.1.9'):
    dochazka = (pd.read_excel(path, sheet_name=sheet, header=3, index_col=0, usecols='A, B, C, E, G, I, K')).iloc[3:-1]
    
    rok_match = re.search(r'(\d{4}-\d{2})', path)
    if rok_match:
        year = rok_match.group(1).replace('-', '/')  # Extrahovaný a upravený školní rok
    else:
        year = "Rok nebyl nalezen"
    
    dochazka.insert(loc=6, column='rok', value=year)
    return dochazka

MS_23_24 = get_MS_dochazka(r'data\CSU\MS2023-24.xlsx')
MS_22_23 = get_MS_dochazka(r'data\CSU\MS2022-23.xlsx')
MS_21_22 = get_MS_dochazka(r'data\CSU\MS2021-22.xlsx')
MS_20_21 = get_MS_dochazka(r'data\CSU\MS2020-21.xlsx')
MS_19_20 = get_MS_dochazka(r'data\CSU\MS2019-20.xlsx')
MS_18_19 = get_MS_dochazka(r'data\CSU\MS2018-19_konkrTab.xlsx', sheet='2300421909')


MS_all = pd.concat([MS_23_24, MS_22_23, MS_21_22, MS_20_21, MS_19_20], ignore_index=False, sort=False)
MS_all = MS_all.rename(columns={'Unnamed: 1' : 'Celkem'})
sloupce = ['Celkem', 'mladší 3 let', '3leté', '4leté', '5leté', '6leté a starší']
MS_all[sloupce] = MS_all[sloupce].astype(int)
# MS_all
# MS_18_19

In [55]:
MS_all

,Celkem,mladší 3 let,3leté,4leté,5leté,6leté a starší,rok
Hlavní město Praha,42771,3187,11365,12719,13562,1938,2023/24
Středočeský kraj,52957,3707,13755,15642,16782,3071,2023/24
Jihočeský kraj,23216,2543,5810,6452,6867,1544,2023/24
Plzeňský kraj,19496,1565,4965,5611,6195,1160,2023/24
Karlovarský kraj,8402,911,2220,2418,2537,316,2023/24
...,...,...,...,...,...,...,...
Kraj Vysočina,17965,2421,4637,4931,4984,992,2019/20
Jihomoravský kraj,41796,4271,11050,11898,12162,2415,2019/20
Olomoucký kraj,22931,3436,5913,5914,6068,1600,2019/20
Zlínský kraj,19999,2737,5154,5438,5268,1402,2019/20


In [56]:
MS_all.to_csv(r'output_files\MS_all_dochazka.csv', mode='w')

### Počet cizinců ve školkách
Počet cizinců v MŠ je již zahrnut MS_all

1) údaje o fyzických osobách (každé dítě je evidováno jen pod jedním státním občanstvím, pokud má dítě dvojí občanství, upřednostní se české, dále občanství státu EU)

In [57]:
dochazka_cizinci = (pd.read_excel(r'data\CSU\MS2023-24.xlsx', sheet_name='1.1.15', header=3, index_col=0, usecols='A, H:L')).drop(labels='Česká republika')
dochazka_cizinci

,2019/20,2020/21,2021/22,2022/23,2023/24
Hlavní město Praha,4701,4805,4893,6222,6258
Středočeský kraj,1816,1793,1891,3022,3104
Jihočeský kraj,485,470,474,959,931
Plzeňský kraj,804,807,837,1338,1324
Karlovarský kraj,489,457,394,670,612
Ústecký kraj,633,625,631,1197,1150
Liberecký kraj,479,478,441,795,755
Královéhradecký kraj,342,319,295,722,634
Pardubický kraj,370,332,332,676,632
Kraj Vysočina,222,225,211,529,577
